In [173]:
!pip install geojson

In [199]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd
import json
import geojson
from geojson import Feature, Point, FeatureCollection, dump

### Load agenda contents

In [200]:
f = urllib.request.urlopen('https://www.bredajazzfestival.nl/locatie/')
contents = f.read()
soup = BeautifulSoup(contents)

In [201]:
venues = soup.find_all(class_='event-content')

In [202]:
def place_location(name):
    if name == '01. Riant-Stoutenbeek podium - Grote Markt Midden': return Point((4.775947, 51.588270))
    if name == '02. Scorpions Security podium - Grote Markt zuid': return Point((4.776352, 51.587545))
    if name == '03. Meinselect podium - St Janstraat': return Point((4.777113, 51.587659))
    if name == '04. De Rooi Pannen podium - Pleintje Veemarktstraat': return Point((4.778927, 51.588654))
    if name == '05. Landgoed de Klokkenberg podium - Willem Merkxtuin': return Point((4.779095, 51.589469))
    if name == '06. Avans Talent podium - Kerkplein': return Point((4.774779, 51.589225))
    if name == '07. ’t Sas terras': return Point((4.774774, 51.588515))
    if name == '08. Van Iersel Luchtman Advocaten podium - \'t Sas Pleintje': return Point((4.774109, 51.588308))
    if name == '09. RB-Media podium - Havermarkt': return Point((4.773685, 51.589236))
    if name == '10. Winters podium - Spanjaardsgat': return Point((4.773433, 51.590183))
    if name == '11. Thomassen Gifts podium - Haven': return Point((4.773170, 51.589743))
    if name == '12. Van Mossel podium - Tolbrug': return Point((4.771347, 51.588376))
    if name == '13. Podium BAAI': return Point((4.770584, 51.593107))
    if name == '14. Podium T-Huis': return Point((4.780578, 51.592023))
    if name == '15. Ingenhousz Tuinterras': return Point((4.768646, 51.587514))
    if name == '16. NS station en Willemstraat': return Point((4.779627, 51.593312))
    if name == '17. Golfpark De Haenen podium - Hotel Nassau': return Point((4.773269, 51.586951))
    if name == '18. Vrienden van Breda Jazz Festival podium - De Avenue': return Point((4.774843, 51.586666))
    if name == '19. Publieke Werken': return Point((4.778373, 51.589056))
    if name == '20. Zuylen podium - Grote Kerk': return Point((4.775542, 51.589011))
    if name == '21. Holy Moly': return Point((4.774906, 51.588664))
    if name == '22. De Graanbeurs': return Point((4.775667, 51.589687))
    if name == '23. MotMot Gallery': return Point((4.772272, 51.593624))
    if name == 'Binnenstad': return Point((4.775344, 51.586513))
    raise Exception('not supported: ' + name)

In [203]:
band_events = []
for v in venues:
    place_name = v.find('h3').text
    events = v.find_all(class_='each-event')
    for e in events:
        band_name = e.find('p').text
        event_url = e.find('a')['href']
        # print('>> EVENT:')
        # print(band_name)
        # print(event_url)
        # print(venue_name)
        band_events.append({
            'band_name': band_name,
            'event_url': event_url,
            'place_name': place_name,
            'event_location': place_location(place_name)
        })

### Add band details

In [204]:
def mon(name):
    if name == 'mei': return '05'
    raise Exception('not supported: ' + name)

In [205]:
for be in band_events:
    url = be['event_url']
    print('loading ' + url)
    f = urllib.request.urlopen(url)
    contents = f.read()
    soup = BeautifulSoup(contents)
    
    burl = soup.find('a', class_='artist-tag')
    if burl is not None:
        band_page_url = burl['href']
        be['band_page_url'] = band_page_url

    edate = soup.find('div', class_='date').find('span').text
    dparts = edate.split(' ')
        
    etime = soup.find('div', class_='time').find('span').text
    tparts = etime.split(' - ')

    be['start'] = dparts[2] + '-' + mon(dparts[1]) + '-' + dparts[0] + 'T' + tparts[0] + ':00'
    be['end'] = dparts[2] + '-' + mon(dparts[1]) + '-' + dparts[0] + 'T' + tparts[1] + ':00'

    description = soup.find('div', class_='ticket_details').find_all('p')
    descriptionStr = ''
    for desc in description:
        descriptionStr = descriptionStr + desc.text + '\n'
    be['band_description'] = descriptionStr
    
    iurl = soup.find('img', alt='artist image')    
    if iurl is not None:
        band_image_url = iurl['src']
        be['band_image_url'] = band_image_url
#         break

loading https://www.bredajazzfestival.nl/event/opening-breda-jazz-festival-2023/
loading https://www.bredajazzfestival.nl/event/bosco-feat-johnny-rosenberg/
loading https://www.bredajazzfestival.nl/event/calvin-johnson-native-son/
loading https://www.bredajazzfestival.nl/event/andrew-bahams-new-orleans-tribute-to-louis-armstrong/
loading https://www.bredajazzfestival.nl/event/soul-brass-band/
loading https://www.bredajazzfestival.nl/event/sons-of-freuds-wife/
loading https://www.bredajazzfestival.nl/event/gregory-agid-presents/
loading https://www.bredajazzfestival.nl/event/jeffrey-miller-band-ft-miles-lyons/
loading https://www.bredajazzfestival.nl/event/louis-armstrong-celebration-band/
loading https://www.bredajazzfestival.nl/event/shea-pierre-tribute-to-ellis-marsalis/
loading https://www.bredajazzfestival.nl/event/steve-walker-traditional-jazzband/
loading https://www.bredajazzfestival.nl/event/satchmocracy-3/
loading https://www.bredajazzfestival.nl/event/traditional-jazznight-wi

loading https://www.bredajazzfestival.nl/event/cinelli-brothers/
loading https://www.bredajazzfestival.nl/event/the-funketeers-feat-miko-weaver/
loading https://www.bredajazzfestival.nl/event/nieuwe-veste-bigband/
loading https://www.bredajazzfestival.nl/event/amsterdamse-studenten-bigband/
loading https://www.bredajazzfestival.nl/event/john-boutte-tribute-to-sam-cooke/
loading https://www.bredajazzfestival.nl/event/philip-lassiter/
loading https://www.bredajazzfestival.nl/event/the-roaring-cats/
loading https://www.bredajazzfestival.nl/event/tollak-ollestad/
loading https://www.bredajazzfestival.nl/event/deltaphonic/
loading https://www.bredajazzfestival.nl/event/bosco-feat-dirty-harrie/
loading https://www.bredajazzfestival.nl/event/vlijmscherp-feat-licks-and-brains/
loading https://www.bredajazzfestival.nl/event/all-hands-on-deck-farewell-concert-2/
loading https://www.bredajazzfestival.nl/event/the-cinelli-brothers-2/
loading https://www.bredajazzfestival.nl/event/the-hi-stakes/
lo

### Show sample

In [206]:
pevents = pd.DataFrame(band_events)
pevents

,band_name,event_url,place_name,event_location,start,end,band_description,band_image_url,band_page_url
0,Opening Breda Jazz Festival 2023,https://www.bredajazzfestival.nl/event/opening...,01. Riant-Stoutenbeek podium - Grote Markt Midden,"{'type': 'Point', 'coordinates': [4.775947, 51...",2023-05-18T13:00:00,2023-05-18T13:30:00,Het Breda Jazz Festival gaat vanzelfsprekend v...,https://www.bredajazzfestival.nl/wp-content/up...,NaN
1,BOSCO feat. Johnny Rosenberg,https://www.bredajazzfestival.nl/event/bosco-f...,01. Riant-Stoutenbeek podium - Grote Markt Midden,"{'type': 'Point', 'coordinates': [4.775947, 51...",2023-05-18T13:30:00,2023-05-18T15:00:00,Jazz en pop orkest BOSCO werd in 2003 opgerich...,https://www.bredajazzfestival.nl/wp-content/up...,https://www.bredajazzfestival.nl/artiest/bosco...
2,Calvin Johnson & Native Son,https://www.bredajazzfestival.nl/event/calvin-...,01. Riant-Stoutenbeek podium - Grote Markt Midden,"{'type': 'Point', 'coordinates': [4.775947, 51...",2023-05-18T16:00:00,2023-05-18T18:00:00,Maak je klaar om de goede tijden te laten roll...,https://www.bredajazzfestival.nl/wp-content/up...,https://www.bredajazzfestival.nl/artiest/calvi...
3,Andrew Baham's New Orleans Tribute to Louis Ar...,https://www.bredajazzfestival.nl/event/andrew-...,01. Riant-Stoutenbeek podium - Grote Markt Midden,"{'type': 'Point', 'coordinates': [4.775947, 51...",2023-05-18T19:00:00,2023-05-18T21:00:00,Sluit je aan bij Andrew Baham en zijn getalent...,https://www.bredajazzfestival.nl/wp-content/up...,https://www.bredajazzfestival.nl/artiest/andre...
4,SOUL Brass Band,https://www.bredajazzfestival.nl/event/soul-br...,01. Riant-Stoutenbeek podium - Grote Markt Midden,"{'type': 'Point', 'coordinates': [4.775947, 51...",2023-05-18T22:00:00,2023-05-18T00:00:00,Soul Brass Band brengt het feest uit New Orlea...,https://www.bredajazzfestival.nl/wp-content/up...,https://www.bredajazzfestival.nl/artiest/soul-...
...,...,...,...,...,...,...,...,...,...
207,LEMBO,https://www.bredajazzfestival.nl/event/lembo/,23. MotMot Gallery,"{'type': 'Point', 'coordinates': [4.772272, 51...",2023-05-18T13:30:00,2023-05-18T15:00:00,Op zoek naar een ontspannen sfeer om te relaxe...,https://www.bredajazzfestival.nl/wp-content/up...,https://www.bredajazzfestival.nl/artiest/lembo/
208,Brass Bands From The World,https://www.bredajazzfestival.nl/event/brass-b...,Binnenstad,"{'type': 'Point', 'coordinates': [4.775344, 51...",2023-05-18T14:30:00,2023-05-18T19:15:00,"Bands die op straat spelen, al dan niet met ee...",https://www.bredajazzfestival.nl/wp-content/up...,NaN
209,Brass Bands From The World,https://www.bredajazzfestival.nl/event/brass-b...,Binnenstad,"{'type': 'Point', 'coordinates': [4.775344, 51...",2023-05-19T13:00:00,2023-05-19T18:00:00,"Bands die op straat spelen, al dan niet met ee...",https://www.bredajazzfestival.nl/wp-content/up...,https://www.bredajazzfestival.nl/artiest/brass...
210,Brass Bands From The World,https://www.bredajazzfestival.nl/event/brass-b...,Binnenstad,"{'type': 'Point', 'coordinates': [4.775344, 51...",2023-05-20T13:00:00,2023-05-20T18:00:00,Brass Bands From The World is een wereldmuziek...,https://www.bredajazzfestival.nl/wp-content/up...,NaN


In [207]:
# json_contents = json.dumps(band_events)

# with open('output.json', 'w') as f:
#     f.write(json_contents)

### Create geojson

In [208]:
event_features = []

for be in band_events:
    be2 = be.copy()
    del be2['event_location']
    event_features.append(Feature(geometry=be['event_location'], properties=be2))

In [209]:
feature_collection = FeatureCollection(event_features)

with open('output.geojson', 'w') as f:
    dump(feature_collection, f)